In [4]:
import numpy as np
import sympy as sym
import matplotlib as plt
from sympy import init_session
init_session()
a,b,c, p,q,r,s,t = symbols('a b c p q r s t', integer=True)

μ,π,σ, λ= symbols('mu pi sigma lambda')

arr = lambda x: np.array(x)
rat  = lambda x : sym.Rational(x)
fact = lambda x : sym.factorial(x)

IPython console for SymPy 1.9 (Python 3.9.7-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.9/



In [5]:
Ω = 10000
#X = [5,25,100]
X = arr(list(range(1,6)))
nX = [200,20,5]
#PX = get_X()#[r(0.14),r(0.19),r(0.26),]#6*[Rational(1,6)]
fx = 2*exp(-2*x)
#PX, X, fx


In [6]:
get_nΩ = lambda: float(input('Enter size of Ω'))
get_X =  lambda: [rat(i)  for i in input('data for X as list e.g prize money').split(',')]
get_nX = lambda: arr([float(i)  for i in input('data for n of X as list e.g no of prizes(winning tickets)').split(',')])
get_PX = lambda : [float(i) if i.isnumeric() ==True else sym.sympify(i) for i in input('input P(x) as float or fraction').split(',')]
#Defines an inputted function fx 
get_fx =  lambda :  sym.parse_expr(input('Enter function'))

#finds P(X=x) given length Ω and n of X 
PX_givΩX = lambda nX, Ω: [Rational(i, Ω) for i in nX]


#finds P(X=x) given a function f(x)
PX_givfx = lambda X :  [fx(i) for i in X] 

#F(x) = P(X≤x) = Σf(x)
pmf = lambda fx: sum(fx)

#F(x) = P(X≤x) = ∫f(x)dx
pdf = lambda fx,a,b :sym.integrate(fx,(x,a,b))

#n!/x!(n-x)!
C = lambda n, x : fact(n) / (fact(x)*fact(n-x))

#Σa<b f(x)
Σ = lambda a, b, fx : sum([i*fx for i in range(a,b+1)])

#E[X=x] = Σx*P(X=x) = Σx*f(x)
E_Xdisc = lambda X, PX: sum(np.array(X)*np.array(PX)) 

E_Xcont = lambda X, fx,a,b: sym.integrate(X*fx,(x,a,b))

E_XplusY = lambda X,PX,Y,PY : E_Xdisc(X,PX) + E_Xdisc(Y,PY)

#var[x] = E[X**2] - (E[X])**2
sqre = lambda X: [i**2 for i in X] if type(X) == list else X**2
#D,C = 'D','C'
Var_disc = lambda X, PX: E_Xdisc(arr(X)*arr(X),PX) - sqre(E_Xdisc(X,PX))  
Var_cont = lambda X, fx,a,b: sym.integrate(sqre(X - E_Xcont(x,fx,a,b))*fx,(x,a,b))

Xstd = lambda X, μ, σ: (X - μ )/ σ

σ_var = lambda X, PX: sym.sqrt(Var_disc(X, PX))
μ_np = lambda X,PX: len(X)*[E_Xdisc(X,PX)]
μ_r =lambda X,PX,r: sum(((X-µ(X,PX))**r)*PX)

# nCx * p^x * q^n-x
Xbin = lambda p,n,x: C(n,x) * p**x * (1-p)**(n-x)
Xbin_ab = lambda p,n,a,b: sum([Xbin(p,n,i) for i in range(a,b+1)])

Xbin_μ = lambda p,n: n*p
Xbin_var = lambda p,n : n*p*(1-p)
Xbin_σ = lambda p,n: np.sqrt(Xbin_var(p,n)) #√n*p*(1-p))

Xbin_stats = lambda p,n: print(f'μ = {Xbin_μ(p,n)},    σ = {Xbin_σ(p,n) },     var = σ^2 = {Xbin_var(p,n)}')

# (λ^x * e^-k )/ k!
Xpoi = lambda λ, x : ((λ**x) * exp(-λ)) / fact(x) if isinstance(x, (int, float, sym.Symbol) )  else sum([Xpoi(λ,i) for i in range(x[0],x[1]+1)])
#Xpoi_ab = lambda λ,a,b: sum([Xpoi(λ,i) for i in range(a,b+1)]) 

Xpoi_stats = lambda λ :  print(f'μ = {λ},    σ = {sym.sqrt(λ)},     var = σ^2 = {λ}')

# q^x * p
Xgeo = lambda p, x: ((1-p)**(x-1))*p if isinstance(x, (int, float, sym.Symbol) )else sum([Xgeo(p,i) for i in range(x[0],x[1]+1)])
Xgeo_stats = lambda p:  print(f'μ = {(1-p)/p},    σ = {sym.sqrt((1-p)/p**2)},     var = σ^2 = {(1-p)/p**2}')

#(e^-(((x-μ)^2) / (2*σ^2)) ) / σ * 2 * √2π
Xnorm = lambda μ, σ, a,b: round(sym.integrate((exp(-(1/2)*(((x-μ)**2) / (σ**2 )))  / (σ * sym.sqrt(2*np.pi))) , (x,a,b)),5)
Z = lambda a,b : Xnorm(0,1,a,b)

#M_xt



#Least sqaures
#lSquarVal = lambda X , Y : 
lSquarMat = lambda X,Y :  np.vstack(( #np.hstack(( 
                                   
                                                X, Y, arr(
                                                            np.around([[i[0]**2, i[0]*i[1] , i[1]**2 ] for i in list(zip(X,Y))], 3).T)
                                                    ))
                                        
                                        #,arr([['X','Y','X^2','XY','Y^2']]).T
                                    #))

ΣSqr_mat = lambda X,Y :  np.around([sum(i) for i in lSquarMat(X,Y)], 3)


In [9]:
Xpoi(5.5,3), Xpoi(5.5,2), Xpoi(5.5,1), Xpoi(5.5,0), Xpoi(5.5,3)+Xpoi(5.5,2)+Xpoi(5.5,1)+Xpoi(5.5,0)

In [12]:
λ

In [13]:
#sym.integrate(λ**x) * exp(-λ)

In [14]:
#2.1,2.7,3.1,3.2,3.6,4.2
#1.6,1.8,2.4,2.5,2.7,3.1
#4.7,4.8,4.9,5.0,5.1]
#1.2,1.4,1.5,1.5,1.7
X = arr([1,2,3,5])
Y = arr([2.99,5.82,9.14,15.12])

arr(['X','Y','X^2','XY','Y^2']).T
X,Y, list(zip(X,Y)), 
#print(lSquarVal(X,Y)) ,
lSquarMat(X,Y),ΣSqr_mat(X,Y)#arr((X,Y))

(array([[  1.   ,   2.   ,   3.   ,   5.   ],
        [  2.99 ,   5.82 ,   9.14 ,  15.12 ],
        [  1.   ,   4.   ,   9.   ,  25.   ],
        [  2.99 ,  11.64 ,  27.42 ,  75.6  ],
        [  8.94 ,  33.872,  83.54 , 228.614]]),
 array([ 11.   ,  33.07 ,  39.   , 117.65 , 354.966]))

In [15]:
Σx ,Σy,Σx2, Σxy, Σy2 = ΣSqr_mat(X,Y)
xbar = Σx / len(X)
ybar = Σy / len(Y)

β1 = round((Σxy - ybar * Σx )/ (Σx2 - xbar *Σx),3)
β0 = round((ybar - β1 * xbar),3)

line_yValues = lambda β0, β1 ,x: β0 + β1* x


Δy = Y -line_yValues(β0, β1 ,X)
d = lambda  Δy: np.sqrt(Δy**2)

#y_valLine = 

Σsqr_ε = sum(d(Δy)**2)
print(f'Σx={Σx}  ,Σx2={Σx2}  ,Σxy={Σxy}  ,\nxbar={xbar}  ,ybar={ybar}, \nΣy2={Σy2}  ,Σy={Σy},\nβ0={β0}  , β1={β1}  ')
#X,Y,Σx,Σy,Σxy,Σx2, xbar, ybar, β0, β1,Δy, d(Δy),Σsqr_ε


Σx=11.0  ,Σx2=39.0  ,Σxy=117.65  ,
xbar=2.75  ,ybar=8.2675, 
Σy2=354.966  ,Σy=33.07,
β0=-0.126  , β1=3.052  


In [16]:
SSt = Σy2 - len(Y)*ybar**2
SSx = sum([(i - xbar)**2 for i in X])
SSr = β1**2 * SSx
SSe = SSt-SSr
MSe = SSe/(len(X)-2)
To = np.sqrt(SSr/MSe)

SSt, SSx, SSr, SSe, MSe,To

In [17]:
[(i - xbar)**2 for i in X]

In [18]:
#(Xnorm(150,40,120,190)).evalf(), 
Xnorm(0,1,1,.3)
Xnorm(72,4,71,75)# Xnorm(75,7,96,1000)
Xnorm(72,4,69.32,-oo)
Xnorm(21,7,12.02838,29.97162)
Z(0.1,oo)


In [19]:
Xbin_ab(0.15,10,2,3)
Xbin(0.2,15,3),Xbin_ab(0.2,15,3,15)
Xbin_ab(0.2,4,0,1),Xbin_ab(0.2,4,0,4)
Xbin_stats(0.2,4)

#Xbin(0.15,10,5)
Xpoi(4,2), N(Xpoi(4,2))
#Xpoi_ab(4,0,1),N(Xpoi_ab(4,0,1))

#[N(i) for i in [Xpoi(4,3),Xpoi(4,2),Xpoi(4,1),Xpoi(4,0)]] 
[[N(i+j) for j in reversed([Xpoi(4,3),Xpoi(4,2),Xpoi(4,1),Xpoi(4,0)])] for i in [Xpoi(4,3),Xpoi(4,2),Xpoi(4,1),Xpoi(4,0)]] 
[N(j) for j in reversed([Xpoi(4,3),Xpoi(4,2),Xpoi(4,1),Xpoi(4,0)])]
Xpoi(3.3, 1 )
Xpoi(3.3, (1,30 )), #Xpoi_ab(3.3,1,30)
N(Xpoi(8,3))
Xpoi(6.5,9)
Xpoi(6.5,(0,3))

p=.75
sum([Xgeo(p, 1),Xgeo(p, 2), Xgeo(p, 3)])#, Xgeo(p, 0 )])
#Xgeo_stats(p)

#((1-p)**x)*p, Xgeo(.75, 1)
Xgeo(p,(1,2))

μ = 0.8,    σ = 0.8,     var = σ^2 = 0.6400000000000001


In [20]:
'''X=get_nX()#5.1,4.9,4.7,4.6,5,5.4
Y=get_nX()#1.4,1.4,1.3,1.5,1.4,1.7'''
#2.1,2.7,3.1,3.2,3.6,4.2
#1.6,1.8,2.4,2.5,2.7,3.1
Σx = sum(X)
Σy = sum(Y)
Σxy = np.dot(X,Y)
Σx2 = sum(sqre(X))
xbar = Σx / len(X)
ybar = Σy / len(Y)
X,Y,
Σx,Σy,Σxy,Σx2, #xbar, ybar, β0, β1,Δy, d(Δy),Σsqr_ε

(11, 33.07, 117.65, 39)

In [21]:
Y**2

array([  8.9401,  33.8724,  83.5396, 228.6144])

In [22]:
line_x = np.linspace(min(X),max(X),100)

#y = map(line_y(β0,β1, line_x), line_x)
#line_y(β0,β1, line_x)
plt.scatter(X,Y)
plt.plot(line_x,line_yValues(β0,β1, line_x))
plt.show()

QSocketNotifier: Can only be used with threads started with QThread
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [25]:
Ω = 10000
#X = [5,25,100]
X = get_X()#[-1,0,1/2,1]#arr(list(range(1,6)))
nX = [200,20,5]
PX = get_X()#[r(0.14),r(0.19),r(0.26),]#6*[Rational(1,6)]
fx = 2*exp(-2*x)
PX, X, fx

In [26]:
((1-a)**(x-1))*a

In [27]:
4*0.2*(1-0.2)#sqrt(0.8**2)

In [28]:

n, x = 10 , 3
fact(n), fact(x) , fact(n-x) ,C(10,3)

In [29]:
X = get_X();X

In [30]:
μ_r(X,1/3,r)

TypeError: 'Symbol' object is not callable

In [ ]:
iter(x) == True
#sym.integrate(sqre(x - E_Xcont(x,fx,a,b))*fx,(x,a,b))
#Var_cont(x,fx,0,oo)
#Xstd (78,72,4)

TypeError: 'Symbol' object is not iterable

In [ ]:
X,sqre(X),sqre(x)

In [ ]:
(X-µ(X,PX))**2+

array([-2.14, -1.14, -0.14,  0.86,  1.86])

In [ ]:

#PXgivfx(X())
#EXdisc(X(),PXgivΩX(X()))

#E_XplusY(list(range(1,7)),PX,list(range(1,7)),PX)
E_Xdisc(sqre(X), PX),Var_disc(X,PX), σ(X,PX),oo
#?sym.integrate
#X
#fx = get_fx()
#fx

In [ ]:
fx

<function _lambdifygenerated(x)>

In [ ]:
list(range(1,7))
sym.sympify(X)
Rational(3,5)


In [ ]:
[i for i in PX]

In [ ]:
EXdisc()
#mom = lambda X :

TypeError: <lambda>() missing 2 required positional arguments: 'X' and 'PX'

In [ ]:
sizeΩ

10000

In [ ]:
PX()

TypeError: 'list' object is not callable